In [1]:
# ============================================================================
# STEP 1: Clone Repository & Install Dependencies
# ============================================================================

# Commented out IPython magic to ensure Python compatibility.
# Clone the repository
!git clone https://github.com/pranavacchu/capstone-BLIP.git
%cd capstone-BLIP

# Install dependencies
print("📦 Installing dependencies... This will take 3-5 minutes")
!pip install -q opencv-python-headless pillow numpy pandas tqdm python-dotenv
!pip install -q torch torchvision transformers sentence-transformers
!pip install -q pinecone FlagEmbedding

print("\n✅ Installation complete!")

# Check GPU availability
import torch

if torch.cuda.is_available():
    print(f"\n🚀 GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️ No GPU detected. Using CPU (slower but works)")

Cloning into 'capstone-BLIP'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 316 (delta 46), reused 54 (delta 20), pack-reused 231 (from 1)
Receiving objects: 100% (316/316), 387.26 KiB | 9.45 MiB/s, done.
Resolving deltas: 100% (199/199), done.
/content/capstone-BLIP
📦 Installing dependencies... This will take 3-5 minutes
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ============================================================================
# STEP 2a: Check CLIP Model Availability
# ============================================================================
# Only tries the known compatible model for image embeddings

print('Checking available CLIP model candidates...')
from embedding_generator import load_image_model

candidates = [
    'clip-ViT-B-32',  # Known compatible model
]

image_model = load_image_model(candidates)
if image_model:
    print("✅ Successfully loaded an image model.")
else:
    print("❌ No compatible image model could be loaded. Please check your configuration.")

Checking available CLIP model candidates...


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

0_CLIPModel/model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Successfully loaded an image model.


In [3]:
# ============================================================================
# STEP 2b: Set CLIP Model & Runtime Config Overrides
# ============================================================================

from video_search_config import Config

# Use a CLIP model id known to work for image embeddings with sentence-transformers.
# Recommended: 'clip-ViT-B-32' (do NOT use 'openai/clip-vit-base-patch32')
Config.CLIP_MODEL_NAME = 'clip-ViT-B-32'
print('Notebook override: Config.CLIP_MODEL_NAME =', Config.CLIP_MODEL_NAME)

Notebook override: Config.CLIP_MODEL_NAME = clip-ViT-B-32


# STEP 3: Install Dependencies & Clone Repository
# This section handles repo setup and dependency installation.

In [4]:
# ============================================================================
# STEP 3 (Optional): Install Object Detection Dependencies (Grounding DINO)
# ============================================================================
# Skip this if you don't need object detection; standard BLIP captioning is faster.

print("Installing Grounding DINO and dependencies...")
print("This may take 2-3 minutes...")
import subprocess

!pip install -q supervision timm
subprocess.run(['pip', 'install', '-q', 'timm'], check=False)
subprocess.run(['pip', 'install', '-q', 'supervision'], check=False)

print("\nGrounding DINO dependencies installed!")
print("Models will be downloaded automatically from Hugging Face on first use")

Installing Grounding DINO and dependencies...
This may take 2-3 minutes...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 8.2 MB/s eta 0:00:00

Grounding DINO dependencies installed!
Models will be downloaded automatically from Hugging Face on first use


# STEP 4: Configure Pinecone & Environment
# Set your Pinecone API key and connection details here.

# STEP 4a: Set Pinecone API Credentials

In [5]:
# ============================================================================
# STEP 4a: Configure Pinecone (Serverless)
# ============================================================================

from pinecone import Pinecone, ServerlessSpec

# IMPORTANT: Replace this with your NEW serverless API key (starts with pcx_)
PINECONE_API_KEY = "pcsk_7FUgsS_6sCNFNzFNYMGAe5USrrGR8Pr69XbfbV6s7v3cb23AgHNsY8kGY9CGRuwGbHL5yd"

# Write to .env file
with open('.env', 'w') as f:
    f.write(f"PINECONE_API_KEY={PINECONE_API_KEY}\n")

print("✅ Pinecone serverless configuration saved!")

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create index if not exists
INDEX_NAME = "video-index"

if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print("🆕 Created serverless index:", INDEX_NAME)
else:
    print("ℹ️ Index already exists:", INDEX_NAME)

# Get index reference
index = pc.Index(INDEX_NAME)
print("📌 Pinecone index ready:", INDEX_NAME)

✅ Pinecone serverless configuration saved!
ℹ️ Index already exists: video-index
📌 Pinecone index ready: video-index


# STEP 5 (Optional): Start API Server for React Integration

Run this cell ONLY if you want to use the React frontend.

This cell:
- Authenticates with ngrok (requires free account at https://ngrok.com)
- Starts a FastAPI server that runs on Colab's GPU
- Creates a public URL you can use from your React app

If you just want to use the notebook directly (without React), you can skip the next code cell.

In [6]:
# Install FastAPI and dependencies (only needed once per runtime)
!pip install -q fastapi uvicorn pyngrok nest-asyncio python-multipart

from pyngrok import ngrok

# Set your ngrok authtoken (get it from: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "your_ngrok_token_here"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok authenticated successfully!")

print("⏭️  Skipping React API server setup by default.")
print("   To enable the React frontend:")
print("   1. Set NGROK_AUTH_TOKEN above")
print("   2. Uncomment ngrok.set_auth_token and install lines")
print("   3. Use the FastAPI + ngrok server defined in colab_api_server.py")

✅ ngrok authenticated successfully!
⏭️  Skipping React API server setup by default.
   To enable the React frontend:
   1. Set NGROK_AUTH_TOKEN above
   2. Uncomment ngrok.set_auth_token and install lines
   3. Use the FastAPI + ngrok server defined in colab_api_server.py


# STEP 5 (Optional): Start React API Server for Frontend Integration
# Skip if you only want to use the notebook. Keep running in background if using React UI.

In [ ]:
# Install FastAPI and dependencies
!pip install -q fastapi uvicorn pyngrok nest-asyncio python-multipart

# Authenticate ngrok
from pyngrok import ngrok

# Set your ngrok authtoken (get it from: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "2uq19SSTYxlDhPoiMXnJXHuKAUV_6w77sQcS4KX3fRnDr1c4W"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok authenticated successfully!")

# Start the API server with ngrok tunnel
import nest_asyncio
import uvicorn
from colab_api_server import app

nest_asyncio.apply()

print("\n🌐 Starting ngrok tunnel on port 8000...")
print("=" * 80)

# Kill any existing tunnels
ngrok.kill()

# Create ngrok tunnel
public_url_obj = ngrok.connect(8000, bind_tls=True)
# Extract the public URL string
public_url = str(public_url_obj).split('"')[1] if '"' in str(public_url_obj) else public_url_obj.public_url

print(f"\n✅ COLAB API SERVER READY")
print("=" * 80)
print(f"\n🌍 Public URL: {public_url}")
print(f"\n📋 Copy this URL and paste it into your React app:")
print(f"   {public_url}")
print("\n" + "=" * 80)

# Initialize the video search engine
print("\n🔧 Initializing Video Search Engine...")
from video_search_engine import VideoSearchEngine

# Create a global engine instance for the app to use
import colab_api_server
if not colab_api_server.engine:
    colab_api_server.engine = VideoSearchEngine()
    print("✅ Engine initialized successfully!")
else:
    print("✅ Engine already initialized!")

# =========================================================================
# 🚀 PRE-LOAD EMBEDDING MODEL (Avoids 2.24GB download on first search)
# =========================================================================
print("\n🔄 Pre-loading embedding model...")
print("   This will download ~2.24GB (one-time, takes 2-3 minutes)")
print("   Future searches will be FAST!")

try:
    from embedding_generator import EmbeddingGenerator
    temp_embedder = EmbeddingGenerator()
    # Trigger model download and initialization
    _ = temp_embedder.generate_embedding("warmup")
    print("✅ Embedding model ready! Searches will now be fast.")
except Exception as e:
    print(f"⚠️  Model pre-load failed: {e}")
    print("   First search will still work but will be slower")

# =========================================================================

# Display GPU status
import torch
if torch.cuda.is_available():
    print(f"\n🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️  Running on CPU (slower)")

print("\n🚀 Starting FastAPI server...")
print("   Keep this notebook running while using the React app")
print("   The server will process videos using Colab's GPU")
print("\n" + "=" * 80)
print("\n📡 Server is ready to accept requests!")
print("   Go to your React app and paste the URL above to connect")
print("\n" + "=" * 80 + "\n")

# Start the server (this will block and keep running)
# Python 3.12+ compatible approach - run server directly with asyncio
import asyncio

async def start_server():
    """Start the FastAPI server using uvicorn"""
    config = uvicorn.Config(
        app,
        host="0.0.0.0",
        port=8000,
        log_level="info"
    )
    server = uvicorn.Server(config)
    await server.serve()

# Run the server
asyncio.run(start_server())

✅ ngrok authenticated successfully!

🌐 Starting ngrok tunnel on port 8000...

✅ COLAB API SERVER READY

🌍 Public URL: https://5435eabeb198.ngrok-free.app

📋 Copy this URL and paste it into your React app:
   https://5435eabeb198.ngrok-free.app


🔧 Initializing Video Search Engine...
✅ Engine initialized successfully!

🔄 Pre-loading embedding model...
   This will download ~2.24GB (one-time, takes 2-3 minutes)
   Future searches will be FAST!
⚠️  Model pre-load failed: cannot import name 'EmbeddingGenerator' from 'embedding_generator' (/content/capstone-BLIP/embedding_generator.py)
   First search will still work but will be slower

🚀 GPU: Tesla T4
   Memory: 15.8 GB

🚀 Starting FastAPI server...
   Keep this notebook running while using the React app
   The server will process videos using Colab's GPU


📡 Server is ready to accept requests!
   Go to your React app and paste the URL above to connect




INFO:     Started server process [1405]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Initializing Video Search Engine...
✅ Video Search Engine initialized successfully
📊 GPU Status:
   ✓ GPU Available: Tesla T4
   ✓ GPU Memory: 15.8 GB
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/health HTTP/1.1" 200 OK
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "GET /api/health HTTP/1.1" 200 OK
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/upload HTTP/1.1" 200 OK
📤 Uploading: Untitled video - Made with Clipchamp (online-video-cutter.com) (1).mp4 -> ec67ccc6_Untitled video - Made with Clipchamp (online-video-cutter.com) (1).mp4
✅ Upload complete: 23.15 MB
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "POST /api/upload HTTP/1.1" 200 OK
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/process?video_filename=ec67ccc6_Untitled%20video%20-%20Made%20with%20Clipchamp%20(online-video-cutter.com)%20(1).mp4 HTTP/1.1" 200 OK
📋 Job a760c85c-6181-415b-9a38-1218d1fb28a8 queued for processing
INFO:     2405:201:d005:d199:

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

...

   ✓ Uploaded 7 vectors
   Sample caption: Laptop: An A laptop computer sitting on top of a table....

📁 Namespace: videos:2025-11-25:person_man
   Uploading 5 vectors...


Uploading to Pinecone: 100%|██████████| 1/1 [00:00<00:00,  2.28it/s]


   ✓ Uploaded 5 vectors
   Sample caption: Laptop: An A woman sitting on a couch using a laptop....

📁 Namespace: videos:2025-11-25:desk
   Uploading 4 vectors...


Uploading to Pinecone: 100%|██████████| 1/1 [00:00<00:00,  2.63it/s]


   ✓ Uploaded 4 vectors
   Sample caption: Person Woman: An a man in a red shirt and blue jeans....

✅ UPLOAD COMPLETE: 16 vectors uploaded for 'Untitled video - Made with Clipchamp (online-video-cutter.com) (1).mp4'
   Sample vector IDs: e6d072b6ba7b_obj0_combined, e6d072b6ba7b_obj1_combined, e6d072b6ba7b_obj2_combined...


✅ Job a760c85c-6181-415b-9a38-1218d1fb28a8 completed in 120.3 seconds
   Frames: 19
   Embeddings: 16

INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/job/a760c85c-6181-415b-9a38-1218d1fb28a8 HTTP/1.1" 200 OK
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/job/a760c85c-6181-415b-9a38-1218d1fb28a8 HTTP/1.1" 200 OK
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/job/a760c85c-6181-415b-9a38-1218d1fb28a8 HTTP/1.1" 200 OK
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/job/a760c85c-6181-415b-9a38-1218d1fb28a8 HTTP/1.1" 200 OK
INFO:     2405:201:d005:d199:39aa:c9e6:f1bc:3937:0 - "OPTIONS /api/job

In [ ]:
# ============================================================================
# STEP 6: Upload Video File
# ============================================================================
# Choose how to provide your video: upload from computer, URL, or YouTube

from google.colab import files
import os
import subprocess
from urllib.parse import urlparse, parse_qs

print("📤 Choose how to get your video:\n")
print("1. Upload from computer (recommended for small files < 100MB)")
print("2. Download from URL (direct video file)")
print("3. Download from YouTube URL\n")

choice = input("Enter choice (1/2/3): ").strip()
video_path = None

if choice == "1":
    print("\n📁 Upload your video file...")
    uploaded = files.upload()
    for filename, file_data in uploaded.items():
        video_path = filename
        print(f"✅ Uploaded: {video_path}")
        break

elif choice == "2":
    url = input("Enter video URL: ").strip()
    print(f"\n⬇️  Downloading from {url}...")
    video_path = "downloaded_video.mp4"
    subprocess.run(['wget', '-q', url, '-O', video_path], check=False)
    if os.path.exists(video_path):
        print(f"✅ Downloaded: {video_path}")
    else:
        print("❌ Failed to download")
        video_path = None

elif choice == "3":
    yt_url = input("Enter YouTube URL: ").strip()
    print(f"\n▶️  Downloading from YouTube...")
    # !pip install -q yt-dlp
    video_path = "youtube_video.mp4"
    # subprocess.run(['yt-dlp', '-f', 'best[ext=mp4]', yt_url, '-o', video_path], check=False)
    print(f"✅ Downloaded: {video_path}")

else:
    print("Invalid choice")

# Validate the video file
if video_path:
    if os.path.exists(video_path):
        file_size = os.path.getsize(video_path) / (1024**2)
        print(f"\n✅ Video file found: {video_path}")
        print(f"   Size: {file_size:.2f} MB")
    else:
        print(f"\n❌ File not found: {video_path}")
        video_path = None

if not video_path:
    print("\n⚠️  No video file available. Please upload or download a video first.")

In [ ]:
# =========================================================================
# STEP 7: Initialize Enhanced Components (Captioning, Embeddings, Temporal)
# =========================================================================

print("🔧 INITIALIZING ENHANCED COMPONENTS")
print("=" * 70)

from caption_generator import BlipCaptionGenerator
from embedding_generator import TextEmbeddingGenerator, MultimodalEmbeddingGenerator
from temporal_bootstrapping import TemporalBootstrapper
from video_search_config import Config

# Default settings (can be customized here)
captioning_model = "base"  # or "large", "tiny", "object-focused"
# Enable dual embeddings (caption + image)
enable_dual_embeddings = True
# Enable temporal smoothing of confidences
enable_temporal_bootstrapping = True

# Enable multiple caption variants + CLIP reranking for better quality
generate_captions_variants = True
num_variants = 3

model_map = {
    "base": "Salesforce/blip-image-captioning-base",
    "large": "Salesforce/blip-image-captioning-large",
    "tiny": "Salesforce/blip-image-captioning-tiny",
    "object-focused": "Salesforce/blip-image-captioning-large"
}

# Initialize caption generator
logger_model = model_map.get(captioning_model, "Salesforce/blip-image-captioning-base")
print(f"\n📝 Caption generator: {captioning_model} ({logger_model})")

caption_gen = BlipCaptionGenerator(
    model_name=logger_model,
    model_type=captioning_model,
    batch_size=8,
    use_gpu=True,
    max_length=50,
    num_beams=4,
    generate_multiple_captions=generate_captions_variants,
    captions_per_frame=num_variants,
    compute_confidence=True,
    # Use CLIP-based reranking of caption variants against the image
    enable_clip_rerank=True,
    clip_rerank_model=Config.CLIP_MODEL_NAME
)
print("✅ Caption generator ready")

# Initialize embedding generator
if enable_dual_embeddings:
    print(f"\n🖼️  Multimodal embeddings enabled (caption + image)")
    embedding_gen = MultimodalEmbeddingGenerator(
        caption_model="intfloat/multilingual-e5-large",
        image_model=Config.CLIP_MODEL_NAME or "clip-ViT-B-32",
        batch_size=32,
        use_gpu=True
    )
else:
    print(f"\n📄 Text embeddings only")
    embedding_gen = TextEmbeddingGenerator(
        model_name="intfloat/multilingual-e5-large",
        batch_size=32,
        use_gpu=True
    )
print("✅ Embedding generator ready")

# Initialize temporal bootstrapper
temporal_bootstrapper = None
if enable_temporal_bootstrapping:
    print(f"\n⏱️  Temporal bootstrapping enabled")
    temporal_bootstrapper = TemporalBootstrapper(
        temporal_window=5,
        confidence_threshold=0.5,
        consistency_weight=0.3,
        smoothing_sigma=1.5
    )
    print("✅ Temporal bootstrapper ready")

print("\n" + "=" * 70)
print("✅ All components initialized!")


In [ ]:
# ============================================================================
# STEP 8: Process Video End-to-End
# ============================================================================
# This step:
# 1. Extracts frames from video (removing redundant ones)
# 2. Generates captions using BLIP
# 3. Creates embeddings (caption + image if enabled)
# 4. Computes confidence scores with temporal bootstrapping
# 5. Uploads to Pinecone for semantic search
#
# Expected time: 2-5 minutes per minute of video (with GPU)

from datetime import datetime
from video_search_engine import VideoSearchEngine

if 'video_path' not in locals() or not video_path:
    print("❌ No video file available. Please run STEP 6 first.")
else:
    print("\n" + "=" * 80)
    print("🎬 VIDEO PROCESSING PIPELINE")
    print("=" * 80)
    print(f"Video file: {video_path}")
    print("=" * 80)

    # Initialize engine
    engine = VideoSearchEngine()

    # Get video name and date
    video_name = input("\n📝 Enter a name for this video (e.g., 'camera1_office'): ").strip() or "video_upload"
    video_date = input("📅 Enter video date (YYYY-MM-DD, default=today): ").strip() or datetime.today().strftime("%Y-%m-%d")

    # Force-enable object detection for object-focused captions
    use_object_detection = True
    print("\n🎯 Object detection + captioning is ENABLED by default for this run.")

    # Process video
    print("\n⏳ Processing video (this may take a few minutes)...")
    stats = engine.process_video(
        video_path=video_path,
        video_name=video_name,
        video_date=video_date,
        save_frames=False,
        upload_to_pinecone=True,
        use_object_detection=use_object_detection
    )

    print("\n" + "=" * 80)
    print("✅ VIDEO PROCESSING COMPLETE")
    print("=" * 80)
    print(f"\n� Processing Statistics:")
    print(f"   Total frames extracted: {stats.get('total_frames_extracted', 0)}")
    print(f"   Frames with captions: {stats.get('frames_with_captions', 0)}")
    print(f"   Embeddings uploaded: {stats.get('embeddings_uploaded', 0)}")
    print(f"   Processing time: {stats.get('processing_time_seconds', 0):.1f} seconds")
    print(f"   Frame reduction: {stats.get('frame_reduction_percent', 0):.1f}%")
    print("\n✅ Your video is now indexed and ready for search!")

📤 Choose how to get your video:

1. Upload from computer (recommended for small files < 100MB)
2. Download from URL (direct video file)
3. Download from YouTube URL

Enter choice (1/2/3): 3

Enter YouTube URL (video or shorts): https://www.youtube.com/shorts/QhlroYnundk
⬇️ Downloading from YouTube...
   Installing/updating yt-dlp...
   Fetching video (this may take a minute)...
✅ Downloaded successfully: youtube_video.mp4

📹 Video ready: youtube_video.mp4 (0.7 MB)
   Duration: 7.6 seconds
   FPS: 30.0
   Total frames: 228


# STEP 9: Search Videos Using Natural Language Queries
# Use natural language to find moments in your video!

In [ ]:
# ============================================================================
# STEP 9a: Initialize Enhanced Components (Captioning, Embeddings, Temporal)
#          + Initialize Search Engine & Run First Query
# ============================================================================

print("🔧 INITIALIZING ENHANCED COMPONENTS (for search)")
print("=" * 70)

from caption_generator import BlipCaptionGenerator
from embedding_generator import TextEmbeddingGenerator, MultimodalEmbeddingGenerator
from temporal_bootstrapping import TemporalBootstrapper
from video_search_config import Config
from video_search_engine import VideoSearchEngine

# Default settings (can be customized here)
captioning_model = "base"  # or "large", "tiny", "object-focused"
# Enable dual embeddings (caption + image)
enable_dual_embeddings = True
# Enable temporal smoothing of confidences
enable_temporal_bootstrapping = True

# Enable multiple caption variants + CLIP reranking for better quality
generate_captions_variants = True
num_variants = 3

model_map = {
    "base": "Salesforce/blip-image-captioning-base",
    "large": "Salesforce/blip-image-captioning-large",
    "tiny": "Salesforce/blip-image-captioning-tiny",
    "object-focused": "Salesforce/blip-image-captioning-large"
}

# Initialize caption generator
logger_model = model_map.get(captioning_model, "Salesforce/blip-image-captioning-base")
print(f"\n📝 Caption generator: {captioning_model} ({logger_model})")

caption_gen = BlipCaptionGenerator(
    model_name=logger_model,
    model_type=captioning_model,
    batch_size=8,
    use_gpu=True,
    max_length=50,
    num_beams=4,
    generate_multiple_captions=generate_captions_variants,
    captions_per_frame=num_variants,
    compute_confidence=True,
    # Use CLIP-based reranking of caption variants against the image
    enable_clip_rerank=True,
    clip_rerank_model=Config.CLIP_MODEL_NAME
)
print("✅ Caption generator ready")

# Initialize embedding generator
if enable_dual_embeddings:
    print(f"\n🖼️  Multimodal embeddings enabled (caption + image)")
    embedding_gen = MultimodalEmbeddingGenerator(
        caption_model="intfloat/multilingual-e5-large",
        image_model=Config.CLIP_MODEL_NAME or "clip-ViT-B-32",
        batch_size=32,
        use_gpu=True
    )
else:
    print(f"\n📄 Text embeddings only")
    embedding_gen = TextEmbeddingGenerator(
        model_name="intfloat/multilingual-e5-large",
        batch_size=32,
        use_gpu=True
    )
print("✅ Embedding generator ready")

# Initialize temporal bootstrapper
temporal_bootstrapper = None
if enable_temporal_bootstrapping:
    print(f"\n⏱️  Temporal bootstrapping enabled")
    temporal_bootstrapper = TemporalBootstrapper(
        temporal_window=5,
        confidence_threshold=0.5,
        consistency_weight=0.3,
        smoothing_sigma=1.5
    )
    print("✅ Temporal bootstrapper ready")

print("\n" + "=" * 70)
print("✅ All enhanced components initialized!\n")

# =========================================================================
# Initialize Search Engine & Run First Query (as before)
# =========================================================================

# Check if engine is already initialized from previous steps
if 'engine' not in locals() or engine is None:
    print("[INFO] Initializing VideoSearchEngine for search...")
    engine = VideoSearchEngine()
else:
    print("[INFO] Using existing VideoSearchEngine instance.")

print("🔍 VIDEO SEMANTIC SEARCH")
print("=" * 60)
print("Enter natural language queries to find moments in your video")
print("Examples: 'person walking', 'red object', 'someone talking'\n")

# First search query
query = input("🔍 Enter your search query: ").strip()

if not hasattr(engine, 'search') or not callable(engine.search):
    print("❌ The engine object does not have a working 'search' method. Please check initialization.")
elif not query:
    print("No query provided.")
else:
    print(f"\nSearching for: '{query}'...")
    print("=" * 60)

    try:
        results = engine.search(
            query=query,
            top_k=5,
            similarity_threshold=0.5
        )
        if results:
            print(f"\n✅ Found {len(results)} results:\n")
            for i, result in enumerate(results, 1):
                timestamp = result.get('timestamp', 0)
                time_formatted = result.get('time_formatted', '00:00')
                similarity = result.get('similarity_score', 0)
                caption = result.get('caption', 'N/A')

                print(f"{i}. ⏱️  {time_formatted} (similarity: {similarity:.3f})")
                print(f"   Caption: {caption}")
                if result.get('thumbnail_path'):
                    print(f"   Thumbnail: {result.get('thumbnail_path')}")
                print()
        else:
            print("❌ No results found. Try a different query or ensure videos are indexed.")
    except Exception as e:
        print(f"❌ Search failed: {e}")

Enter a name for this video (or press Enter for auto-name): testfr


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



🎬 Processing video: testfr
📅 Video date: 2025-11-10
⏳ This will take a few minutes... Please wait.



preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

   ✓ Uploaded 2 vectors
   Sample caption: Backpack: An A man with a backpack walking down the street....

📁 Namespace: videos:2025-11-10:folder
   Uploading 1 vectors...


Uploading to Pinecone: 100%|██████████| 1/1 [00:00<00:00,  4.35it/s]


   ✓ Uploaded 1 vectors
   Sample caption: File: An a white refrigerator sitting on top of a wooden floor....

📁 Namespace: videos:2025-11-10:coat_jacket
   Uploading 1 vectors...


Uploading to Pinecone: 100%|██████████| 1/1 [00:00<00:00,  4.48it/s]


   ✓ Uploaded 1 vectors
   Sample caption: Coat Jacket: An A man in a red jacket is walking down the street....

✅ UPLOAD COMPLETE: 6 vectors uploaded for 'testfr'
   Sample vector IDs: e6d072b6ba7b_obj0_emb, 6d8f1a3acd55_obj1_emb, 6d8f1a3acd55_obj2_emb...



✅ VIDEO PROCESSING COMPLETE!

📊 Processing Statistics:
   Video name: testfr
   Video date: 2025-11-10
   Frames extracted: 6
   Frames with captions: 6
   Captions before dedupe: 6
   Unique embeddings: 6
   ✅ Actually uploaded: 6
   Processing time: 1.5 minutes

   Frame reduction: 0.0%


## Step 9: Search Your Video!

Now you can search for content using natural language queries.

**Example queries:**
- "person walking"
- "black bag"
- "someone talking on phone"
- "car driving"
- "red shirt"

The system will return timestamps where that content appears!

In [ ]:
# Single search query with optional namespace filtering
query = input("🔍 Enter your search query: ")

# Optional: Search in specific object namespace
if use_object_detection:
    print("\n📁 Available namespaces:")
    print("   - backpack, bag, duffel_bag")
    print("   - laptop, tablet")
    print("   - helmet, bottle, folder, umbrella")
    print("   - coat_jacket, suitcase_luggage")
    print("   - (leave empty to search all)")

    namespace_filter = input("\nFilter by namespace (optional): ").strip().lower()
else:
    namespace_filter = ""

print(f"\nSearching for: '{query}'...")
if namespace_filter:
    print(f"Namespace filter: {namespace_filter}")
print("=" * 60)

# Perform search with namespace if specified
if namespace_filter and use_object_detection:
    # Search in specific namespace using Pinecone directly
    query_embedding = engine.embedding_generator.encode_query(query)
    search_results = engine.pinecone_manager.query(
        query_vector=query_embedding,
        top_k=5,
        namespace=namespace_filter,
        include_metadata=True
    )

    # Format results
    results = []
    for result in search_results:
        formatted_result = {
            "timestamp": result.timestamp,
            "caption": result.caption,
            "similarity_score": result.score,
            "frame_id": result.frame_id,
            "video_name": result.video_name,
            "time_formatted": engine._format_timestamp(result.timestamp)
        }
        results.append(formatted_result)
else:
    # Standard search across all namespaces
    results = engine.search(
        query=query,
        top_k=5,
        similarity_threshold=0.5
    )

if results:
    print(f"\n✅ Found {len(results)} results:\n")

    for i, result in enumerate(results, 1):
        print(f"{i}. ⏱️ Timestamp: {result['time_formatted']}")
        print(f"   📝 Caption: {result['caption']}")
        print(f"   📊 Confidence: {result['similarity_score']:.1%}")
        print(f"   🎥 Video: {result['video_name']}")
        print()
else:
    print("\n❌ No results found. Try:")
    print("   - Different search terms")
    print("   - More general queries")
    print("   - Lowering the similarity threshold")
    if namespace_filter:
        print(f"   - Searching without namespace filter (currently: {namespace_filter})")

🔍 Enter your search query: bag

📁 Available namespaces:
   - backpack, bag, duffel_bag
   - laptop, tablet
   - helmet, bottle, folder, umbrella
   - coat_jacket, suitcase_luggage
   - (leave empty to search all)

Filter by namespace (optional):  backpack

Searching for: 'bag'...
Namespace filter: backpack

✅ Found 2 results:

1. ⏱️ Timestamp: 00:03.80
   📝 Caption: Backpack: An A person with a backpack on their back.
   📊 Confidence: 81.1%
   🎥 Video: test

2. ⏱️ Timestamp: 00:00.03
   📝 Caption: Backpack: A The back of a backpack on a green field.
   📊 Confidence: 80.0%
   🎥 Video: test



## Step: Test Connection to Pinecone (from main notebook)

Use this step to verify that your Pinecone credentials and index are working before processing videos.

In [ ]:
from video_search_engine import VideoSearchEngine

print("🔌 Connecting to Pinecone...")
engine = VideoSearchEngine()

# Get database stats
stats = engine.get_index_stats()

print("\n✅ Successfully connected to Pinecone!")
print(f"\n📊 Database Statistics:")
print(f"   Index: {stats.get('index_name', 'N/A')}")
print(f"   Total vectors: {stats.get('total_vectors', 0):,}")
print(f"   Dimension: {stats.get('dimension', 'N/A')}")
print(f"   Capacity: {stats.get('capacity', 'N/A')}")

# Optional: Show namespace statistics if available
if stats.get('namespaces'):
    print(f"\n📁 Namespaces:")
    for namespace, ns_info in stats.get('namespaces', {}).items():
        # Handle NamespaceSummary object - access vector_count attribute
        vector_count = ns_info.vector_count if hasattr(ns_info, 'vector_count') else ns_info
        print(f"   - {namespace}: {vector_count:,} vectors")

## Step: View Namespace Statistics (Object Detection Mode)

If you used object detection mode, this shows how vectors are distributed across object namespaces.

In [ ]:
# View namespace distribution
if 'engine' in locals() and use_object_detection:
    stats = engine.get_index_stats()

    print("📊 NAMESPACE DISTRIBUTION")
    print("=" * 60)

    if 'namespaces' in stats and stats['namespaces']:
        print(f"\nTotal vectors: {stats['total_vectors']:,}")
        print(f"\nVectors by object category:")

        # Sort namespaces by vector count
        namespace_items = []
        for namespace, ns_info in stats['namespaces'].items():
            vector_count = ns_info.vector_count if hasattr(ns_info, 'vector_count') else ns_info
            namespace_items.append((namespace, vector_count))

        namespace_items.sort(key=lambda x: x[1], reverse=True)

        for namespace, count in namespace_items:
            # Format namespace name for display
            display_name = namespace.replace('_', ' ').title()
            percentage = (count / stats['total_vectors'] * 100) if stats['total_vectors'] > 0 else 0
            bar_length = int(percentage / 2)  # Scale to 50 chars max
            bar = "█" * bar_length

            print(f"   {display_name:20s} │ {bar:50s} {count:4d} ({percentage:5.1f}%)")
    else:
        print("No namespace data available yet. Process a video first.")

    print("=" * 60)
else:
    print("⚠️ Namespace statistics only available in object detection mode and when the engine is initialized.")

## Step: Date-Based Search (from main notebook)

Search videos from specific dates or date ranges for faster, more targeted results.

In [ ]:
from video_search_engine import VideoSearchEngine

# Ensure engine exists
if 'engine' not in locals() or engine is None:
    print("[INFO] Initializing VideoSearchEngine for date-based search...")
    engine = VideoSearchEngine()

print("📅 DATE-BASED SEARCH")
print("=" * 60)
print("Search moments from a specific recording date.")
print("Example date: 2025-11-10\n")

query = input("🔍 Enter your search query: ").strip()
if not query:
    print("❌ No query provided.")
else:
    date_filter = input("📅 Enter date (YYYY-MM-DD): ").strip()
    namespace_filter = input("(Optional) Namespace filter (e.g., backpack, folder): ").strip().lower()
    namespace_filter = namespace_filter or None

    if not date_filter:
        print("❌ No date provided. Please run again with a valid date.")
    else:
        print(f"\nSearching for '{query}' on date {date_filter}...")
        if namespace_filter:
            print(f"Namespace filter: {namespace_filter}")
        print("=" * 60)

        try:
            results = engine.search(
                query=query,
                top_k=5,
                similarity_threshold=0.5,
                date_filter=date_filter,
                namespace_filter=namespace_filter
            )

            if results:
                print(f"\n✅ Found {len(results)} results:\n")
                for i, result in enumerate(results, 1):
                    print(f"{i}. ⏱️ Timestamp: {result['time_formatted']}")
                    print(f"   📝 Caption: {result['caption']}")
                    print(f"   📊 Confidence: {result['similarity_score']:.1%}")
                    print(f"   🎥 Video: {result['video_name']}")
                    print()
            else:
                print("\n❌ No results found for that date/query.")
        except Exception as e:
            print(f"❌ Date-based search failed: {e}")

## ✅ Workflow Complete

You have successfully set up and used the Video Search Pipeline!

### What Happened

1. **Cloned repo & installed dependencies** — Set up the BLIP and Pinecone stack
2. **Configured Pinecone** — Set API credentials and connected to vector DB
3. **Optional: Launched React API server** — For frontend integration (kept running in background)
4. **Configured enhanced settings** — Selected captioning model, enabled dual embeddings and temporal bootstrapping
5. **Uploaded video** — From computer, URL, or YouTube
6. **Processed video end-to-end** — Extracted frames → captions → embeddings → uploaded to Pinecone
7. **Searched results** — Ran natural language queries using fusion search (text + image indices)

### Key Features Used

✅ **Dual Embeddings**: Both caption (text) and image (CLIP) embeddings stored and queried  
✅ **Fusion Search**: Combines text and image index results with configurable weights  
✅ **Temporal Bootstrapping**: Confidence scoring with temporal smoothing  
✅ **CLIP Dedupe**: Optional semantic frame deduplication (reduces redundancy)  
✅ **Thumbnails**: Saved and indexed in Pinecone metadata for UI display  

### For Next Time

- To reprocess more videos, start from "Step 5: Upload a Video File"
- To run more searches, just re-run the search cell and enter new queries
- To modify settings (model, weights, thresholds), restart and update the enhanced configuration cell

### Backend Architecture

```
Video → Extract Frames → Caption (BLIP) → Embeddings (dual: caption + image)
      → Temporal Bootstrap (confidence) → Prepare Payload
      → Upload to Pinecone (combined + text + image indices)
      → Query-time Fusion → Score Normalization & Ranking → Results
```

Enjoy exploring your videos! 🚀
